### 감정분석
* 텍스트에 숨겨져 있는 저장의 의도(감정, 정보)를 찾아내는 방법
* Opinion mining, Sentiment mining
* 지식기반(knowlege-base) : 사람(전문가)
* 머신러닝기반(machine learning-base) : 대표적으로 긍정/부정
    * text classification(텍스트 분류)

#### 텍스트 분류
* 벡터형태로 변환된 텍스트를 분류하는 방법
* DNN이 나오기 전까지는 SVM 많이 사용
* 최근은 딥러닝(RNN, LSTM) 많이 사용

In [27]:
import pickle
with open('train_text.pk', 'rb') as f:
    train = pickle.load(f)
with open('test_text.pk', 'rb') as f:
    test = pickle.load(f)

In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install wrapt
   

SyntaxError: invalid syntax (<ipython-input-1-bde39277ee01>, line 1)

In [4]:
import sklearn

In [5]:
len(train), len(test)

(150000, 48760)

In [6]:
train[:5]

[(['진짜', '짜증', '네요', '목소리'], '0'),
 (['포스터', '보고', '초딩', '영화', '오버', '연기', '가볍', '구나'], '1'),
 (['너무', '밓었', '추천', '한다'], '0'),
 (['교도소', '이야기', '구먼', '솔직히', '재미', '평점', '조정'], '0'),
 (['사이몬페그',
   '익살',
   '스런',
   '연기',
   '돋보였',
   '영화',
   '스파이더맨',
   '보이',
   '커스틴',
   '던스트',
   '너무나',
   '이뻐',
   '보였'],
  '1')]

In [7]:
test[0]

(['평점', '나쁘', '짜리', '더더욱', '잖아'], '0')

### NSMC 분류
* 텍스트를 벡터화 머신러닝
* pipeline 처리

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import numpy as np

In [9]:
# 데이터 처리(학습/테스트셋 별 x,y)
train_x = [' '.join(t) for t,c in train] # 학습데이터 문제
train_y = [c for t,c in train] # 학습데이터 정답
test_x = [' '.join(t) for t,c in test] # 테스트데이터 문제
test_y = [c for t,c in test] # 테스트데이터 정답

In [10]:
test_x[:5]

['평점 나쁘 짜리 더더욱 잖아',
 '지루 은데 완전 막장',
 '어도 텐데 나와서 심기 불편',
 '음악 주가 최고 음악 영화',
 '진정 쓰레기']

In [11]:
# 파이프라인
model1 = Pipeline([
    ('vector', CountVectorizer()),
    ('model', SVC(kernel='linear'))
])
model2 = Pipeline([
    ('vector', CountVectorizer()),
    ('model', SVC(kernel='sigmoid'))
])
model3 = Pipeline([
    ('vector', CountVectorizer()),
    ('model', MultinomialNB())
])

In [12]:
# model 변수에 파이프라인객체 대입
model = model1

model.fit(train_x, train_y) # 학습 (문제,답)
pred_train = model.predict(train_x) # 예측
train_acc = (pred_train==train_y).mean()
print('학습데이터 정확도:{}'.format(train_acc))

pred_test = model.predict(test_x) # 학습하지 않은 테스트데이터로 예측
test_acc = (pred_test==test_y).mean()
print('테스트데이터 정확도:{}'.format(test_acc))

KeyboardInterrupt: 

In [12]:
# 텍스트로 직접 예측
from eunjeon import Mecab
mecab = Mecab()
model1 = Pipeline([
    ('vector', CountVectorizer(tokenizer=mecab.pos)),
    ('model', SVC(kernel='linear'))
])

In [13]:
def readFile(file):
    with open(file, encoding='utf8') as f:
        doc = [line.replace('\n','').split('\t') for line in f.readlines()]
    return doc[1:]

In [14]:
train_text = readFile('ratings_train.txt')
test_text = readFile('ratings_test.txt')

In [15]:
train_text[0]

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']

In [16]:
train_x = [text[1] for text in train_text]
train_y = [text[2] for text in train_text]

In [17]:
model1.fit(train_x[:1000], train_y[:1000])

Pipeline(steps=[('vector',
                 CountVectorizer(tokenizer=<bound method Mecab.pos of <eunjeon._mecab.Mecab object at 0x000001800B100DF0>>)),
                ('model', SVC(kernel='linear'))])

In [18]:
import numpy as np
test = np.array([
    '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
    '음악이 주가 된, 최고의 음악영화',
    '진정한 쓰레기'
])

In [19]:
pre = model1.predict(test)

In [20]:
pre

array(['0', '1', '0'], dtype='<U1')

In [21]:
model1.predict(['넌 정말 착하고 재밌는 사람이야'])

array(['1'], dtype='<U1')

In [22]:

model1.predict(['넌 정말 쓰레기같은 사람이야'])

array(['0'], dtype='<U1')

### LSTM 모델로 구현

In [23]:
# import
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
# 파라미터
max_words = 35000
max_len = 50
batch_size=128
epochs = 30

In [28]:
# 데이터 전처리
# 토크나이징, 패딩(제로패딩)
import numpy as np

# 문서만 추출
train_x = [doc for doc, _ in train]

# 토크나이징
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

# 변환(LSTM모델의 입력값을 넣기 위해)
x_train = tokenizer.texts_to_sequences(train_x)
y_train = np.array([int(label) for _, label in train])

x_test = tokenizer.texts_to_sequences([doc for doc, _ in test])
y_test = np.array([int(label) for _, label in test])

# 길이를 동일하게 맞추기
x_train = pad_sequences(x_train, maxlen=max_len, padding='pre')
x_test = pad_sequences(x_test, maxlen=max_len, padding='pre')

# 학습가능한 형태로 변환
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [29]:
train_ds

<BatchDataset shapes: ((None, 50), (None,)), types: (tf.int32, tf.int32)>

In [30]:
# 모델 구현
# 클래스로 구현하는 방법
class MyLSTM(tf.keras.Model):
    # 생성자
    def __init__(self):
        # 부모클래스의 생성자 호출
        super().__init__()
        self.emb = Embedding(max_words, 100)
        self.lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2)
        self.dense = Dense(1, activation='sigmoid')
    
    def call(self, x):
        x = self.emb(x)
        x = self.lstm(x)
        x = self.dense(x)
        return x

In [31]:
# 모델객체 선언
model = MyLSTM()

In [32]:
# 모델설정
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [33]:
# Early Stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3, verbose=1)
# ModelCheckpoint
mc = tf.keras.callbacks.ModelCheckpoint('nsmc_bestmodel.cp', monitor='val_loss', mode='min', save_best_only=True)

In [34]:
# 학습
history = model.fit(train_ds, validation_data=test_ds, epochs=epochs, batch_size=batch_size, callbacks=[es, mc])

Epoch 1/30
1172/1172 [==============================] - 343s 286ms/step - loss: 0.4749 - acc: 0.7579 - val_loss: 0.3930 - val_acc: 0.8189
Epoch 2/30
1172/1172 [==============================] - 321s 274ms/step - loss: 0.3551 - acc: 0.8382 - val_loss: 0.3992 - val_acc: 0.8185
Epoch 3/30
1172/1172 [==============================] - 325s 277ms/step - loss: 0.3062 - acc: 0.8625 - val_loss: 0.4280 - val_acc: 0.8150
Epoch 4/30
1172/1172 [==============================] - 324s 277ms/step - loss: 0.2612 - acc: 0.8835 - val_loss: 0.4857 - val_acc: 0.8097
Epoch 00004: early stopping


In [37]:
def predict_func(text):
    import text_module
    text = text_module.preprocessing(text)
    text = tokenizer.texts_to_sequences([text])
    text = pad_sequences(text, maxlen=max_len, padding='pre')
    if model.predict(text) > 0.5:
        print('긍정')
    else :
        print('부정')

In [38]:
predict_func('정말 최고의 영화')

긍정


In [39]:
predict_func('정말 최악의 영화')

부정
